In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import query, job_config, render, register_filter, register_global
import pandas as pd

<IPython.core.display.Javascript object>

In [29]:
properties = {
    "decentralized Adam, ring (three seeds)": {
        "algorithm": "\dpsgd Adam",
        "topology": "ring",
    },
    "decentralized quasi-global Adam, ring (three seeds)": {
        "algorithm": "Quasi-global Adam",
        "topology": "ring",
    },
    "decentralized RelaySum Adam, strip (three seeds)": {
        "algorithm": "Relay-Adam",
        "topology": "chain",
    },
}

In [62]:
results = []
for key, value in pd.read_pickle("data/distilbert_on_agnews_with_01_alpha.pickle").items():
    props = properties[key]
    for seed, values in enumerate(value):
        print(values["averaged top-1 of local models (each on full test dataset)"])
        accuracy = sum(values["averaged top-1 of local models (each on full test dataset)"][-1:])
        results.append({
            **props,
            "seed": seed,
            "metric": "accuracy",
            "value": accuracy,
        })
df = pd.DataFrame(results).set_index(["algorithm", "topology",  "metric"])
df

[0.3294583261013031, 0.5745546817779541, 0.7385234236717224, 0.8102396130561829, 0.8364322781562805, 0.8511145710945129, 0.8607344031333923, 0.8687864542007446, 0.8746484518051147, 0.8783723711967468]
[0.3264843821525574, 0.5296354293823242, 0.7597265839576721, 0.8230312466621399, 0.8462968468666077, 0.8569505214691162, 0.8626458048820496, 0.8675729036331177, 0.8711276054382324, 0.874010443687439]
[0.45139843225479126, 0.7250573039054871, 0.7864010334014893, 0.8154869675636292, 0.8309817910194397, 0.8417942523956299, 0.8501067757606506, 0.8562890887260437, 0.8619478940963745, 0.8663932085037231]
[0.31003645062446594, 0.6053072810173035, 0.7581406235694885, 0.8183125257492065, 0.8451744914054871, 0.8627526164054871, 0.8736979365348816, 0.881460964679718, 0.8869504928588867, 0.8903645873069763]
[0.307638019323349, 0.5592551827430725, 0.788783848285675, 0.8393046855926514, 0.8554531335830688, 0.8645885586738586, 0.870408833026886, 0.8752031326293945, 0.8798515796661377, 0.8824140429496765

seed     value
algorithm         topology metric                  
\dpsgd Adam       ring     accuracy     0  0.878372
                           accuracy     1  0.874010
                           accuracy     2  0.866393
Quasi-global Adam ring     accuracy     0  0.890365
                           accuracy     1  0.882414
                           accuracy     2  0.877050
Relay-Adam        chain    accuracy     0  0.931112
                           accuracy     1  0.931453
                           accuracy     2  0.933487

In [63]:
@register_filter
def percentage(value):
    val = value * 100
    return f"{val:.1f}\%"

def acc(algorithm, topology):
    return df.loc[algorithm, topology, "accuracy"]["value"].array

In [64]:
register_global("acc", acc)

In [65]:
acc("\dpsgd Adam", "ring")

/Users/anonymized/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  import sys


<PandasArray>
[0.8783723711967468, 0.874010443687439, 0.8663932085037231]
Length: 3, dtype: float64

In [82]:
%%template table
\tablefontsize
\centering
\begin{tabularx}{\textwidth}{X l l}
    \toprule
    Algorithm & Topology & Top-1 Accuracy \\
    \cmidrule(lr){1-2}     \cmidrule(lr){3-3}
    Centralized Adam & fully-connected & 94.2\% $\pm$ 0.1\% \\
    \textbf{Relay-Adam} & chain & \Var{acc("Relay-Adam", "chain").mean() | percentage} $\pm$ \Var{acc("Relay-Adam", "chain").std() | percentage} \\
    \dpsgd Adam & ring & \Var{acc("\dpsgd Adam", "ring").mean() | percentage} $\pm$ \Var{acc("\dpsgd Adam", "ring").std() | percentage} \\
    Quasi-global Adam~\citep{lin2021quasiglobal} & ring & \Var{acc("Quasi-global Adam", "ring").mean() | percentage} $\pm$ \Var{acc("Quasi-global Adam", "ring").std() | percentage} \\
    \bottomrule
\end{tabularx}

In [83]:
!mkdir -p generated
with open("generated/bert-ag-news-results.tex", "w") as fp:
    res = render("table")
    fp.write(res)
    print(res)

\tablefontsize
\centering
\begin{tabularx}{\textwidth}{X l l}
    \toprule
    Algorithm & Topology & Top-1 Accuracy \\
    \cmidrule(lr){1-2}     \cmidrule(lr){3-3}
    Centralized Adam & fully-connected & 94.2\% $\pm$ 0.1\% \\
    \textbf{Relay-Adam} & chain & 93.2\% $\pm$ 0.1\% \\
    \dpsgd Adam & ring & 87.3\% $\pm$ 0.6\% \\
    Quasi-global Adam~\citep{lin2021quasiglobal} & ring & 88.3\% $\pm$ 0.7\% \\
    \bottomrule
\end{tabularx}


/Users/anonymized/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  import sys
